In [ ]:
# Importing necessary libraries
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
from zipfile import ZipFile
import io
import urllib3
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import panel as pn
from panel.interact import interact, interactive, fixed, interact_manual
from panel import widgets
pn.extension('plotly')

# Literature
Air quality with covid cases in Mumbai, India
https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2021GH000383


The role of air pollution (PM and NO2) in COVID-19 spread and lethality: A systematic review
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7444490/


In [ ]:
# Disables the warning when downloading
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def zipped_downloader(url):
    """ Downloads the files necessary for the analysis, unzips and creates a pandas dataframe """
    
    response = requests.get(url, auth=HTTPBasicAuth('user', 'pass'), stream=True, verify=False)
    with ZipFile(io.BytesIO(response.content)) as myzip: # Unzips the file
        with myzip.open(myzip.namelist()[0]) as myfile: # Open first file in folder
            df = pd.read_csv(myfile) # Reads csv and creates a pandas dataframe
            return df      

# Download of PM2.5 FRM/FEM Mass (88101) from 'aqs.epa.gov'
PM2_5_Mass_2020 = zipped_downloader('https://aqs.epa.gov/aqsweb/airdata/daily_88101_2020.zip')
PM2_5_Mass_2021 = zipped_downloader('https://aqs.epa.gov/aqsweb/airdata/daily_88101_2021.zip')
# Downloads nytimes covid-19 data
covid19_usa = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv')

In [ ]:
PM2_5_Mass_2021.columns

## Note about covid data
The covid 19 file is updated on a daily basis, if the file isn't reshaped or deleted this shouldn't be a problem, but in case it goes wrong, please download the dataset at this [google drive](https://drive.google.com/file/d/1Po0Uk1t_jk_5NGeW_MQAKyES_jv_t-zk/view?usp=sharing).

In [ ]:
# Select only the columns of interest
PM2_5_Mass_2021 = PM2_5_Mass_2021[['State Name', 'Date Local', 'AQI']]
PM2_5_Mass_2020 = PM2_5_Mass_2020[['State Name', 'Date Local', 'AQI']]
# covid19_usa = covid19_usa.drop(['fips', 'deaths'], axis = 1)
covid19_usa = covid19_usa.drop(columns=['fips'])

In [ ]:
# Set columns to correct data type
PM2_5_Mass_2020['Date Local'] = pd.to_datetime(PM2_5_Mass_2020['Date Local'])
PM2_5_Mass_2021['Date Local'] = pd.to_datetime(PM2_5_Mass_2021['Date Local'])
covid19_usa['date'] = pd.to_datetime(covid19_usa['date'])

In [ ]:
# No missing data for the selected columns in the covid dataset
# Both air quality files have several rows with missing data

# The rows with NaN's are dropped
PM2_5_Mass_2020 = PM2_5_Mass_2020.dropna()
PM2_5_Mass_2021 = PM2_5_Mass_2021.dropna()

In [ ]:
# The missing data was removed without problem
print(covid19_usa.isnull().sum())
print(PM2_5_Mass_2020.isnull().sum())
print(PM2_5_Mass_2021.isnull().sum())

In [ ]:
print(f"Covid, Begin:{covid19_usa['date'].min()}, End:{covid19_usa['date'].max()}")
print(f"PM2.5 Mass 2020, Begin:{PM2_5_Mass_2020['Date Local'].min()}, End:{PM2_5_Mass_2020['Date Local'].max()}")
print(f"PM2.5 Mass 2021, Begin:{PM2_5_Mass_2021['Date Local'].min()}, End:{PM2_5_Mass_2021['Date Local'].max()}")
print('The covid dataset spans from 2020-2022, the PN2.5 mass data is from 2020 until nov 2021')

In [ ]:
# Concatenates the 2020 and 2021 files
PM2_5_Mass_20et21 = pd.concat([PM2_5_Mass_2020, PM2_5_Mass_2021], axis=0)
# Renames the column to date so both dataframes have the same name for the same column
PM2_5_Mass_20et21.rename(columns = {'Date Local':'date', 'State Name':'state'}, inplace=True)

# From the covid dataset,only years 2020 and 2021 are necessary
covid19_usa_20et21 = covid19_usa[(covid19_usa['date'].dt.year >= 2020) & (covid19_usa['date'].dt.year <= 2021)]

In [ ]:
# To quickly take a look at the covid cases and death, the following plots were generated 
def quickliner(df, query):
    """ Add doctring later , y = query, which can be 'cases' or deaths """
    fig = px.line(df, x="date", y=query, title=f'Covid {query} in the USA', color='state')
    return fig

quickliner(covid19_usa_20et21, "cases")

In [ ]:
# quickliner(covid19_usa, "deaths")

Looking at the above plot, a few states immediately stand out, such as California, Texas, Florida and New York. Both of these have the largest number of covid cases and deaths. Considering however the number of states in the USA, this line plot might be useful for a simple glance, however the dataset needs to be further analysed to come to concrete conclusions.

In [ ]:
PM2_5_Mass_20et21

In [ ]:
# To quickly take a look at the covid cases and death, the following plots were generated 
def quickliner(df):
    """ Add doctring later , y = query, which can be 'cases' or deaths """
    fig = px.scatter(df, x="date", y='AQI', title=f'Air quality in USA', color='state')
    return fig

quickliner(PM2_5_Mass_20et21)

The scatter plot for air quality gives close to no information at all. Nevada, Washington and California seem to stand out a little more then the rest.

In [ ]:

def monthCalc_weather(df):
#     Makes list with states and 
    states = PM2_5_Mass_20et21['state'].unique()
    df_final = pd.DataFrame(columns = df.columns)

    for state in states:
        df_state = df[df['state'] == state]
        month_avg = df_state.groupby(pd.PeriodIndex(df_state['date'], freq="M"))['AQI'].mean().reset_index()
        month_avg = month_avg.assign(state=state)
        df_final = df_final.append(month_avg)

    return df_final.reset_index(drop=True)
    
monthlyWeather = monthCalc_weather(PM2_5_Mass_20et21)

def monthCalc_covid(df):
    states = df['state'].unique()
    df_final = pd.DataFrame(columns = df.columns)

    for state in states:
        df_state = df[df['state'] == state]
        month_avg = df_state.groupby(pd.PeriodIndex(df_state['date'], freq="M"))[['cases', 'deaths']].sum().reset_index()
        month_avg = month_avg.assign(state=state)
        df_final = df_final.append(month_avg)

    return df_final.reset_index(drop=True)

monthlyCovid = monthCalc_covid(covid19_usa_20et21)

In [ ]:
# The output of the above functions is in period format, which cannot be plotted. This format cannot be converted
# to datetime, however converting it to a string first does seem to work.
# Weather data
monthlyWeather['date'] = monthlyWeather['date'].astype(str)
monthlyWeather['date'] = pd.to_datetime(monthlyWeather['date'])
# Covid data
monthlyCovid['date'] = monthlyCovid['date'].astype(str)
monthlyCovid['date'] = pd.to_datetime(monthlyCovid['date'])

In [ ]:
# Creates a variable with the merged Covid and Weather datasets
CW_Merge = pd.merge(monthlyWeather, monthlyCovid,  how='left', left_on=['state','date'], right_on = ['state','date'])
# There seem to have appeared NANs in the new variable, after manual inspection, these seem
# to be  values already missing before the merge, thus they will be removed.
# CW_Merge = CW_Merge[CW_Merge.isna().any(axis=1)] # used to view rows with missing values
CW_Merge = CW_Merge[~CW_Merge.isna().any(axis=1)]
# Converts columns into the correct datatype as cases and deaths are both strings at the moment
CW_Merge[['AQI', 'cases', 'deaths']] = CW_Merge[['AQI', 'cases', 'deaths']].apply(pd.to_numeric, errors='coerce')
CW_Merge = CW_Merge.rename(columns={'cases':'covid cases', 'deaths':'covid deaths'})

In [ ]:
# Creates  a list with the states with highest mean AQI's
topAQI = pd.DataFrame(CW_Merge.groupby(['state'])['AQI'].mean().sort_values(ascending=False)[:10])
topAQI_states = list(topAQI.reset_index()['state'])
# Creates  a list with the states with lowest mean AQI's
minAQI = pd.DataFrame(CW_Merge.groupby(['state'])['AQI'].mean().sort_values(ascending=False)[-10:])
minAQI_states = list(minAQI.reset_index()['state'])

In [ ]:
def statePlotter(state, query):
    """ Query can be covid cases or covid deaths """
    df = CW_Merge
    df = df[df['state'] == state]
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=df['date'], 
                   y=df[query], 
                   name=query),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df['date'], 
                   y=df['AQI'], 
                   name="AQI score"),
        secondary_y=True,
    )

    # Add figure title
    fig.update_layout(
        title_text=f"Daily {query} and AQI scores in {state}"
    )

    # Set x-axis title
    fig.update_xaxes(title_text="<b>date<b>")

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Covid cases</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>AQI score</b>", secondary_y=True)

    return fig

# statePlotter('Ohio')

state_list = list(CW_Merge['state'].unique())
query = ['covid cases', 'covid deaths']

select1 = pn.widgets.Select(name='State', options=state_list)
select2 = pn.widgets.Select(name='Query', options=query)
interact(statePlotter, state=select1, query=select2)

# To-do
* Unify the PM2.5 file - OK
* Get cases per month and average AQI values per month - OK
* Select on only available dates (2020 Jan - 2021 Nov) - OK
* Select most and less polluted states - OK
* Find statistical test to observe difference between states of high and low pollution
    * Convert values 
    * Good : 1-50
    * Moderate : 50-100
    * Unhealthy_sens : 101-150
* Plot line chart with pollution and covid cases over the 2 year period for the main states

In [ ]:
# # CW_Merge.loc[(CW_Merge['AQI'] <= 50),'AQI']='Hip-Hop'
# df = CW_Merge
# category = pd.cut(df['AQI'],bins=[0,50,100,150], labels=['Good','Moderate','Unhealthy'])
# category
CW_Merge.info()

In [ ]:
category = pd.cut(CW_Merge.AQI,bins=[0,50,100,150],labels=['Good','Moderate','Unhealthy'])
CW_Merge.insert(3,'AQI_Category_1',category)

In [ ]:
# Creates a larger number of categories, 
category = pd.cut(CW_Merge.AQI,bins=[0,25,50,75,100,125],labels=['Good_l','Good_u',
                                                              'Moderate_l','Moderate_u',
                                                              'Unhealthy_l'])
CW_Merge.insert(3,'AQI_Category_2',category)

In [ ]:
import researchpy as rp
import scipy.stats as stats

df = CW_Merge
# Removing the NaNs as the statistical test doesnt function with them
df = df[~df['covid cases'].isna()] 
# # Gives a summary of the data
print(rp.summary_cont(df['covid cases'].groupby(df['AQI_Category_1'])))

stats.f_oneway(df['covid cases'][df['AQI_Category_1'] == 'Good'],
               df['covid cases'][df['AQI_Category_1'] == 'Moderate'],
               df['covid cases'][df['AQI_Category_1'] == 'Unhealthy'])

In [ ]:
print(rp.summary_cont(df['covid cases'].groupby(df['AQI_Category_2'])))
# One-way ANOVA for relation between number of covid cases and air quality
stats.f_oneway(df['covid cases'][df['AQI_Category_2'] == 'Good_l'],
               df['covid cases'][df['AQI_Category_2'] == 'Good_u'],
               df['covid cases'][df['AQI_Category_2'] == 'Moderate_l'],
               df['covid cases'][df['AQI_Category_2'] == 'Moderate_u'],
               df['covid cases'][df['AQI_Category_2'] == 'Unhealthy_l'])

In [ ]:
fig = px.box(df, x='AQI_Category_2', y='covid cases', points="all", title=f'Boxplot')
fig.show()

In [ ]:
# import plotly.graph_objects as go

# import pandas as pd

# df = CW_Merge
# fig = go.Figure()

# AQI_Cats = ['Good_l','Good_u','Moderate_l','Moderate_u','Unhealthy_l']

# for cat in AQI_Cats:
#     fig.add_trace(go.Violin(x=df['AQI_Category_2'][df['AQI_Category_2'] == cat],
#                             y=df['covid cases'][df['AQI_Category_2'] == cat],
#                             name=cat,
#                             box_visible=True,
#                             meanline_visible=True))

# fig.show()


In [ ]:
import plotly.graph_objects as go

import pandas as pd

def violinist(query):
    """ Query can be 'covid cases'or 'covid deaths'"""
    df = CW_Merge
    fig = go.Figure()

    AQI_Cats = ['Good_l','Good_u','Moderate_l','Moderate_u','Unhealthy_l']
    AQI_Cats_dict = {'Good_l':'Good lower','Good_u':'Good upper',
                'Moderate_l':'Moderate lower','Moderate_u':'Moderate upper','Unhealthy_l':'Unhealthy lower'}

    for cat in AQI_Cats:
        fig.add_trace(go.Violin(x=df['AQI_Category_2'][df['AQI_Category_2'] == cat],
                                y=df[query][df['AQI_Category_2'] == cat],
                                name=AQI_Cats_dict[cat],
                                box_visible=True,
                                meanline_visible=True))

    return fig

violinist('covid cases')

jitter
boxplots pet group
violin plot
multivariate